# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
# Months dictionary
name_to_num = {
     "Jan-2019":1,
     "Feb-2019":2,
     "Mar-2019":3,
     "Apr-2019":4,
     "May-2019":5,
     "Jun-2019":6,
     "Jul-2019":7,
     "Aug-2019":8,
     "Sep-2019":9,
     "Oct-2019":10,
     "Nov-2019":11,
     "Dec-2019":12}

df["issue_d"] = df["issue_d"].apply(lambda x: name_to_num[x])
df["next_pymnt_d"] = df["next_pymnt_d"].apply(lambda x: name_to_num[x])
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,3,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,3,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,3,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,3,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,3,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
# Create the LabelEncoder instance

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Fitting and encoding the columns with the LabelEncoder

le.fit(df["loan_status"])
df["loan_status"] = le.transform(df["loan_status"])

le.fit(df["pymnt_plan"])
df["pymnt_plan"] = le.transform(df["pymnt_plan"])

le.fit(df["initial_list_status"])
df["initial_list_status"] = le.transform(df["initial_list_status"])

le.fit(df["application_type"])
df["application_type"] = le.transform(df["application_type"])

le.fit(df["hardship_flag"])
df["hardship_flag"] = le.transform(df["hardship_flag"])

le.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = le.transform(df["debt_settlement_flag"])

df = pd.get_dummies(df, columns=["home_ownership","verification_status"])

df.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,10500.0,0.1719,375.35,66000.0,3,1,0,27.24,0.0,0.0,...,61987.0,0,0,0,0,0,1,0,1,0
1,25000.0,0.2000,929.09,105000.0,3,1,0,20.23,0.0,0.0,...,49197.0,0,0,0,1,0,0,0,0,1
2,20000.0,0.2000,529.88,56000.0,3,1,0,24.26,0.0,0.0,...,43144.0,0,0,0,1,0,0,0,0,1
3,10000.0,0.1640,353.55,92000.0,3,1,0,31.44,0.0,1.0,...,76506.0,0,0,0,0,0,1,0,0,1
4,22000.0,0.1474,520.39,52000.0,3,1,0,18.76,0.0,1.0,...,20000.0,0,0,0,1,0,0,1,0,0


## Split the Data into Training and Testing

In [7]:
# Create our features
X = df.drop(columns=["loan_status"])

# Create our target
y = df["loan_status"].values.reshape(-1, 1)

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,...,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,...,6.881700e+04,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,1.726172,0.0,21.778153,0.217766,0.497697,12.587340,...,5.572240e+04,0.0,0.0,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001
std,10277.348590,0.048130,288.062432,1.155800e+05,0.743862,0.0,20.199244,0.718367,0.758122,6.022869,...,5.095845e+04,0.0,0.0,0.095914,0.499311,0.308793,0.479314,0.499520,0.483865,0.355104
min,1000.000000,0.060000,30.890000,4.000000e+01,1.000000,0.0,0.000000,0.000000,0.000000,2.000000,...,1.270000e+02,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,1.000000,0.0,13.890000,0.000000,0.000000,8.000000,...,2.288000e+04,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,2.000000,0.0,19.760000,0.000000,0.000000,11.000000,...,4.200000e+04,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,2.000000,0.0,26.660000,0.000000,1.000000,16.000000,...,7.249900e+04,0.0,0.0,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,3.000000,0.0,999.000000,18.000000,5.000000,72.000000,...,1.426964e+06,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [13]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

acc_score = accuracy_score(y_test, predictions)
acc_score

0.9951176983435048

In [20]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

cm_df

,Predicted 0,Predicted 1
Actual 0,29,63
Actual 1,21,17092


In [21]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.58      0.32      1.00      0.41      0.56      0.29        92
          1       1.00      1.00      0.32      1.00      0.56      0.34     17113

avg / total       0.99      1.00      0.32      0.99      0.56      0.34     17205



In [25]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08873607990425199, 'last_pymnt_amnt'),
 (0.07823540214423783, 'total_rec_int'),
 (0.07438205827851774, 'total_pymnt'),
 (0.07126704533491852, 'total_pymnt_inv'),
 (0.06134589636170975, 'total_rec_prncp'),
 (0.01985036112917305, 'out_prncp_inv'),
 (0.018907498983179137, 'mo_sin_old_rev_tl_op'),
 (0.0187609175827346, 'out_prncp'),
 (0.01702778046635097, 'installment'),
 (0.016919009768326422, 'dti'),
 (0.01603639426232511, 'revol_bal'),
 (0.015641229935960607, 'max_bal_bc'),
 (0.015608741376279158, 'total_rec_late_fee'),
 (0.01553973406627395, 'mo_sin_old_il_acct'),
 (0.015171093745113612, 'bc_util'),
 (0.013502883475544096, 'tot_cur_bal'),
 (0.01328585634623298, 'bc_open_to_buy'),
 (0.012872171544920884, 'annual_inc'),
 (0.012836211854958122, 'loan_amnt'),
 (0.012792192418157544, 'total_bc_limit'),
 (0.012737610781057476, 'avg_cur_bal'),
 (0.012609174254048488, 'tot_hi_cred_lim'),
 (0.012107118785004378, 'total_bal_ex_mort'),
 (0.01206265115528219, 'all_util'),
 (0.01197905319451770

### Easy Ensemble Classifier

In [36]:
# Train the Classifier
from sklearn.ensemble import GradientBoostingClassifier

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=learning_rate,
        max_features=10,
        max_depth=3,
        random_state=0)
    model.fit(X_train_scaled,y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        model.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(
        model.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.997
Accuracy score (validation): 0.995

Learning rate:  0.1
Accuracy score (training): 0.998
Accuracy score (validation): 0.995

Learning rate:  0.25
Accuracy score (training): 0.997
Accuracy score (validation): 0.995

Learning rate:  0.5
Accuracy score (training): 0.995
Accuracy score (validation): 0.994

Learning rate:  0.75
Accuracy score (training): 0.995
Accuracy score (validation): 0.994

Learning rate:  1
Accuracy score (training): 0.994
Accuracy score (validation): 0.993



In [37]:
# Model With a learning rate of 0.1

model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_features=10,
    max_depth=3,
    random_state=0)

# Fit the model
model.fit(X_train_scaled,y_train.ravel())

GradientBoostingClassifier(max_features=10, random_state=0)

In [39]:
# Calculated the balanced accuracy score
predictions = model.predict(X_test_scaled)
accuracy_score(y_test, predictions)

0.9947108398721302

In [40]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,25,67
Actual 1,24,17089


In [41]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.51      0.27      1.00      0.35      0.52      0.25        92
          1       1.00      1.00      0.27      1.00      0.52      0.29     17113

avg / total       0.99      0.99      0.28      0.99      0.52      0.29     17205



### Final Questions

1. Which model had the best balanced accuracy score?
    The Balanced random forest classfier model seems to give a better balanced accuracy score.
    
2. Which model had the best recall score?

    The Balanced random forest classfier model seems to give a better recall score.

3. Which model had the best geometric mean score?

    The balanced random forest classifier model has the better geometric mean score.

4. What are the top three features?

    The top three features are last_pymnt_amnt,total_rec_int, and total_pymnt.